<a href="https://colab.research.google.com/github/wjyChina/Template-For-Deep-Learning/blob/master/FFNN_binary_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import time
import numpy as np  #linear algebra
import pandas as pd #Only CSV IO
import os
import re
from torch.utils import data #dataloader of batch
import torch
import torch.nn as nn
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
from sklearn import metrics
from torch.autograd import Variable #We can ask require grad or not
from torch.optim.optimizer import Optimizer
from torch.optim import lr_scheduler
import torch.nn.functional as F
from torchvision import transforms
import matplotlib.pyplot as plt
import gc
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from keras.utils import np_utils

In [0]:
### Mount Google drive to gain access to data in it ###
from google.colab import drive
drive.mount('/content/drive')

In [0]:
### Change here ###
### Mount the folder of your data. You can use !ls to check all the folders in your current path ###
os.chdir("drive/My Drive/Colab Notebooks/") 
!ls

In [0]:
### Change here ###
### Read in your data ###
data=pd.read_csv('')

In [0]:
### Change here ###
### Split your data by data_y. First you need to split the original data into input and target ###
data_x=
data_y=
x_train,x_test,y_train,y_test = train_test_split(data_x,data_y,test_size=0.25,random_state=0)

In [0]:
x_train=np.array(x_train)
y_train=np.array(y_train)
x_test=np.array(x_test)
y_test=np.array(y_test)

In [0]:
### Change here ###
### Hyperparameters for training ###
seed = 1
learning_rate = 0.001
batch_size = 1024
n_epochs = 10
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [0]:
### Change here ###
class DP(torch.nn.Module):

  def __init__(self):
    super(MLP, self).__init__()
          
    self.linear_1 = nn.Linear(722, 526)
    self.linear_2 = nn.Linear(526, 258)
    self.linear_3 = nn.Linear(258,56)
    self.linear_out = nn.Linear(56, 1)
        
        
  def forward(self, x):
    out = self.linear_1(x)
    # print(out)
    out = torch.tanh(out)
    out = F.dropout(out,p=0.5,training=self.training)
    out = self.linear_2(out)
    out = torch.tanh(out) 
    out = F.dropout(out,p=0.5,training=self.training)
    out = self.linear_3(out)
    out = torch.sigmoid(out) 
    out = F.dropout(out,p=0.5,training=self.training)     
    logits = self.linear_out(out)
    # print(logits)
    return logits

In [0]:
### Change here if you are going to use loss function other than BCEWithLogitsLoss ###
def compute_epoch_loss(model, data_loader):
  curr_loss, num_examples = 0., 0
  loss_fn=nn.BCEWithLogitsLoss()
  with torch.no_grad():
    for features, targets in data_loader:
      features = features.view(-1, 722).to(device)   ### Change here for different features number: 722 ###
      targets = targets.to(device)
      logits = model.forward(features)
      loss = loss_fn(torch.squeeze(logits),targets)
      num_examples += targets.size()[0]
      curr_loss += loss*targets.size()[0]         ### BCEloss is an average loss. ###

    curr_loss = curr_loss / num_examples
    return curr_loss

    
def compute_accuracy(model, data_loader):
  correct_pred, num_examples = 0, 0
  l=0
  with torch.no_grad():
    for features, targets in data_loader:
      features = features.view(-1, 722).to(device)   ### Change here for different features number: 722 ###
      targets = targets.to(device)
      targets = torch.squeeze(targets)
      logits = model.forward(features)
      predicted_labels = torch.sigmoid(logits)
      num_examples += targets.size(0)
      predicted_labels = torch.squeeze(predicted_labels)
      correct_pred += (((np.array(predicted_labels.cpu())>0.5).astype(int) == np.array(targets.cpu())).sum())
    # print(correct_pred)
    return correct_pred/num_examples * 100

In [0]:
trainfoldx = torch.tensor(x_train, dtype=torch.float32).to(device)
trainfoldy = torch.tensor(y_train, dtype=torch.float32).to(device)
x_val_fold = torch.tensor(x_test, dtype=torch.float32).to(device)
y_val_fold = torch.tensor(y_test[:,np.newaxis], dtype=torch.float32).to(device)

train = torch.utils.data.TensorDataset(trainfoldx,trainfoldy)
valid = torch.utils.data.TensorDataset(x_val_fold, y_val_fold)

train_loader = torch.utils.data.DataLoader(train , batch_size=batch_size,shuffle=True)
valid_loader = torch.utils.data.DataLoader(valid, batch_size=batch_size, shuffle=False)

In [0]:
### Train ###
mbatch_cost = []
start_time = time.time()
minibatch_cost = []
torch.manual_seed(seed)
model = DP()
model.to(device)
optimizer = torch.optim.Adam(model.parameters(),lr=0.001, weight_decay=1e-8)  ### Change here for different optimizer ###
loss_fn = torch.nn.BCEWithLogitsLoss()                     ### Change here for different loss function ###
for epoch in range(n_epochs):
    start = time.time()
    model.train()
    grand_loss = 0
    for  batch_idx, (x_batch, y_batch) in enumerate(train_loader):
        logits = model(x_batch)
        loss=loss_fn(torch.squeeze(logits),y_batch)

        #backprop
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        mbatch_cost.append(loss.item())
        if not batch_idx % 50:                    ### Change the number '50' if you want a different interval of output ###
            print ('Epoch: %03d/%03d | Batch %03d/%03d | Loss: %.4f' 
                %(epoch+1, n_epochs, batch_idx, len(train_loader), loss.item()))

                  #evaluation          
    model.eval()
    cost = compute_epoch_loss(model, train_loader)
    # cost_test = compute_epoch_loss(model, valid_loader)
    # epoch_cost.append(cost)

    train_accuracy = compute_accuracy(model, train_loader)
    valid_accuracy = compute_accuracy(model, valid_loader)

    print('Epoch: %03d/%03d Train Cost: %.4f' % (
            epoch+1, n_epochs, cost))
    print('Train Accuracy: %.3f | Test Accuracy: %.3f' % (train_accuracy, valid_accuracy))
    print('Time elapsed: %.2f min' % ((time.time() - start_time)/60))

print('Total Training Time: %.2f min' % ((time.time() - start_time)/60))

In [0]:
### Plot minibatch v.s. loss ###
plt.plot(mbatch_cost)
plt.show()

In [0]:
### Get prediction and model judgement metrics ###
logits = model(x_val_fold)
predicted_prob = torch.sigmoid(logits)
fpr, tpr, thresholds = metrics.roc_curve(y_test, np.array(predicted_prob.detach().cpu()))
plt.plot(fpr, tpr, 'b')
plt.plot([0, 1], [0, 1], 'r--')
plt.xlim([0, 1])
plt.ylim([0, 1])
plt.ylabel('TP')
plt.xlabel('FP')
plt.show()
AUC = metrics.auc(fpr, tpr)
print('AUC=',AUC)
from sklearn.metrics import confusion_matrix
pred=(np.array(predicted_prob.detach().cpu())>0.5).astype(int)  ### Change your shreshold here ###
con=confusion_matrix(y_test,pred)
accuracy=(con[0,0]+con[1,1])/(con[0,1]+con[1,0]+con[0,0]+con[1,1])
precision=con[0,0]/(con[0,0]+con[1,0])
recall=con[0,0]/(con[0,0]+con[0,1])
print('accuracy=',accuracy)
print('error=',1-accuracy)
print('precision=',precision)
print('recall=',recall)